<a href="https://colab.research.google.com/github/panthamramesh/Qualcomm-DL-Hackathon/blob/master/DL_Hackathon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/Prashant-AV/Qualcomm-DL-Hackathon.git

Cloning into 'Qualcomm-DL-Hackathon'...
remote: Enumerating objects: 10, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 10 (delta 1), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (10/10), 30.68 MiB | 19.21 MiB/s, done.
Resolving deltas: 100% (1/1), done.


In [2]:
import os
import zipfile

In [3]:
import zipfile
import os
import shutil

def extract_files_only(zip_file_paths, destination_folder):
    # Create the destination folder if it doesn't exist
    if not os.path.exists(destination_folder):
        os.makedirs(destination_folder)

    for zip_file_path in zip_file_paths:
        with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
            for member in zip_ref.namelist():
                # Check if the member is a file (not a directory)
                if not member.endswith('/'):
                    # Extract the file to a temporary location
                    source = zip_ref.open(member)
                    target_path = os.path.join(destination_folder, os.path.basename(member))
                    with open(target_path, "wb") as target:
                        shutil.copyfileobj(source, target)

# Example usage
zip_file_paths = ['/content/Qualcomm-DL-Hackathon/train/images part-1.zip', '/content/Qualcomm-DL-Hackathon/train/images part-2.zip']
destination_folder = '/content/Qualcomm-DL-Hackathon/Images'

extract_files_only(zip_file_paths, destination_folder)
print(f"Files from {zip_file_paths} have been extracted to {destination_folder}.")

Files from ['/content/Qualcomm-DL-Hackathon/train/images part-1.zip', '/content/Qualcomm-DL-Hackathon/train/images part-2.zip'] have been extracted to /content/Qualcomm-DL-Hackathon/Images.


In [4]:
def count_files_in_folder(folder_path):
    with os.scandir(folder_path) as entries:
        return sum(1 for entry in entries if entry.is_file())

# Example usage
folder_path = '/content/Qualcomm-DL-Hackathon/Images'
file_count = count_files_in_folder(folder_path)
print(f"Number of files in the folder: {file_count}")

Number of files in the folder: 2352


In [5]:
import os
import pandas as pd
from PIL import Image
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

In [6]:
# Set device (GPU if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [7]:
# Paths to the CSV files and image folder
train_csv_path = '/content/Qualcomm-DL-Hackathon/train/train.csv'
test_csv_path = '/content/Qualcomm-DL-Hackathon/test.csv'
image_folder_path = '/content/Qualcomm-DL-Hackathon/Images'

# Data transformations
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
])

# Load datasets
def load_data(csv_file, root_dir, transform):
    data_frame = pd.read_csv(csv_file)
    images = []
    labels = []
    for idx in range(len(data_frame)):
        img_name = os.path.join(root_dir, data_frame.iloc[idx, 0])
        image = Image.open(img_name)
        if transform:
            image = transform(image)
        images.append(image)
        if 'emergency_or_not' in data_frame.columns:
            labels.append(data_frame.iloc[idx, 1])
        else:
            labels.append(-1)
    return images, labels

train_images, train_labels = load_data(train_csv_path, image_folder_path, transform)
test_images, test_labels = load_data(test_csv_path, image_folder_path, transform)

train_data = list(zip(train_images, train_labels))
test_data = list(zip(test_images, test_labels))

train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)

In [16]:
for i,j in train_data:
  print(i)

Streaming output truncated to the last 5000 lines.
         [0.4784, 0.4863, 0.4980,  ..., 0.3647, 0.3490, 0.3412]],

        [[0.3529, 0.3451, 0.3451,  ..., 0.0314, 0.0431, 0.0471],
         [0.3608, 0.3569, 0.3490,  ..., 0.0431, 0.0471, 0.0510],
         [0.3804, 0.3804, 0.3686,  ..., 0.0549, 0.0510, 0.0471],
         ...,
         [0.4314, 0.4392, 0.4510,  ..., 0.2902, 0.2941, 0.3020],
         [0.4314, 0.4392, 0.4510,  ..., 0.3020, 0.3059, 0.2863],
         [0.4314, 0.4392, 0.4510,  ..., 0.3137, 0.2980, 0.2902]]])
tensor([[[0.7922, 0.8824, 0.8980,  ..., 0.1451, 0.0980, 0.0941],
         [0.6588, 0.6745, 0.7451,  ..., 0.1098, 0.0863, 0.0706],
         [0.7490, 0.7608, 0.7373,  ..., 0.0941, 0.0745, 0.0588],
         ...,
         [0.4392, 0.4431, 0.4510,  ..., 0.4941, 0.4549, 0.4392],
         [0.4431, 0.4510, 0.4588,  ..., 0.6902, 0.6510, 0.6392],
         [0.4314, 0.4471, 0.4706,  ..., 0.6941, 0.6863, 0.6941]],

        [[0.7686, 0.8627, 0.8745,  ..., 0.1451, 0.0980, 0.0941],
     

In [8]:
def create_model():
    model = nn.Sequential(
        nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2, padding=0),
        nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2, padding=0),
        nn.Flatten(),
        nn.Linear(64 * 32 * 32, 512),
        nn.ReLU(),
        nn.Linear(512, 2)  # Assuming 2 classes: emergency and non-emergency
    )
    return model

model = create_model()

In [9]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [17]:
num_epochs = 10

def train_model(model, train_loader, criterion, optimizer, num_epochs):
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for images, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}')

train_model(model, train_loader, criterion, optimizer, num_epochs)

Epoch [1/10], Loss: 0.1921
Epoch [2/10], Loss: 0.0995
Epoch [3/10], Loss: 0.0561
Epoch [4/10], Loss: 0.0572
Epoch [5/10], Loss: 0.0318
Epoch [6/10], Loss: 0.0222
Epoch [7/10], Loss: 0.0226
Epoch [8/10], Loss: 0.0140
Epoch [9/10], Loss: 0.0166
Epoch [10/10], Loss: 0.0198


In [18]:
def evaluate_model(model, test_loader):
    model.eval()
    predictions = []
    with torch.no_grad():
        for images, _ in test_loader:
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            predictions.extend(predicted.cpu().numpy())
    return predictions

predictions = evaluate_model(model, test_loader)

# Save predictions to CSV
test_df = pd.read_csv(test_csv_path)
test_df['emergency_or_not'] = predictions
test_df.to_csv('predictions.csv', index=False)

print("Predictions saved to predictions.csv")

Predictions saved to predictions.csv
